# CRM Data Cloud Modeling

The goal of this notebook is to design a comprehensive model of the entities and relations captured in the CDC. It's loaded from the Data Base and converted into a python-accessible class.

In [32]:
# Library Import

import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,time,math,itertools,os,sys

import pandas as pd

import snowflake.connector

## Connect to Snowflake

Build connection framework for pulling data from Snowflake.

In [33]:
conn = snowflake.connector.connect(
    user='jan-lucas.deinhard@siemens-healthineers.com',
    account='shsitdl.west-europe.azure',
    authenticator='externalbrowser'
)

## Select a subset of CKA accounts of interest.

We restrict this to an examination of a selected subset of CKA accounts first, in order to reduce model size.

In [34]:
# Construct query
accID_query_ = '''
SELECT DISTINCT "ACCOUNT_ID" 
FROM CRMCL_TOPICAREA.OSC."Accounts" 
WHERE "ACCOUNT_ID (MainAccount)" IN (
SELECT DISTINCT TOP 5 "ACCOUNT_ID (MainAccount)" 
FROM CRMCL_TOPICAREA.OSC."Accounts" WHERE "CKAP (Flag)" =\'Y\'
)
'''

# Run query
acc_subset_ = '\''+'\',\''.join([str(k) for k in pd.read_sql(accID_query_,con=conn)['ACCOUNT_ID'].tolist()])+'\''

## Select other entities
* Further accounts info
* Opptys
* OLIs
* Leads
* Vitro Quotes
* Vivo Quotes
* Services Quotes
* Assets IB
* SDTB IB
* GCR Revenue Data
* Orders

### Accounts

In [35]:
acc_sql = '''
SELECT
"ACCOUNT_ID",
"ACCOUNT_ID (MainAccount)",
"ACCOUNT_ID (Parent)",
"Account Address",
"Account Country",
"Account Name",
"Account Name (MainAccount)",
"Account Postal Code",
"Blocking Status",
"CKAP (Flag)",
"Is Partner (Flag)",
"Legal Structure",
"OSC Account Number",
"RKAP (Flag)",
"SAP-ID",
"SAP-ID In Vitro"
FROM CRMCL_TOPICAREA.OSC."Accounts" 
WHERE "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
acc_frame = pd.read_sql(acc_sql,con=conn)

### Opportunities

In [36]:
oppty_sql = '''
SELECT
"ACCOUNT_ID",
"Calculated Expected Turnover Date (earliest)",
"Calculated Expected Turnover Date (latest)",
"Calculated Revenue Start Date (earliest)",
"Calculated Revenue Start Date (latest)",
"Calculated Shipment Date (earliest)",
"Calculated Shipment Date (latest)",
"Direct/Partner Business",
"Indirect Business Type",
"OLI.Close Date (earliest)",
"OLI.Close Date (latest)",
"OPPORTUNITY_ID",
"OPPORTUNITY_ID (Alternate Opportunity)",
"OPPORTUNITY_ID (Linked Opportunity)",
"Opportunity Expired (Flag)",
"Oppty Name",
"Oppty Number",
"Oppty Type",
"Oppty.Close Date",
"Oppty.Created Timestamp",
"Oppty.Forecast Status (Consolidated)",
"Oppty.IT/Automation Included (Flag)",
"Oppty.Lease End Date",
"Oppty.Project Progress",
"Oppty.Quote Number",
"Oppty.Status (Consolidated)",
"Part of ES Project (Flag)",
"Sale Type (Oppty Level)",
"Sales Program (Oppty Level)"
FROM CRMCL_TOPICAREA.OSC."Opportunities" 
WHERE "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
oppty_frame = pd.read_sql(oppty_sql,con=conn)

### Opportunity Line Items

In [37]:
oli_sql = '''
SELECT
"ACCOUNT_ID",
"ACCOUNT_ID (End User)",
"Alternate (Flag)",
"Calculated Expected Turnover Date",
"Calculated Revenue Start Date",
"Calculated Shipment Date",
"Capital Funnel Value (GC)",
"Contract Duration (Months)",
"Contract Effective Funnel Value (GC)",
"DQ Issue (Comment)",
"DQ Issue (Flag)",
"Decision Date",
"Expected Delivery Date",
"Expected Installation Date",
"Expected Shipment Date",
"Expected Turnover Date",
"Expired (Flag)",
"FV Units",
"Finance Type",
"Funnel Status",
"Funnel Value (GC)",
"Funnel Value (LC)",
"Ghost (Flag)",
"Include in Forecast (Flag)",
"Incumbent Vendor",
"Lease End Date",
"Lost w FC (Flag)",
"Monthly Recurring Funnel Value (GC)",
"Multiple Quotes attached (Flag)",
"OLI.Business Area",
"OLI.Business Line",
"OLI.Close Date",
"OLI.Created Timestamp",
"OLI.Quantity",
"OLI.Status",
"OLI.Status (Grouped)",
"OLI_ID",
"OPPORTUNITY_ID",
"Prior Year Revenue Run Rate (GC)",
"Product Number",
"Recurring Expected (Flag)",
"Recurring Revenue Funnel (GC)",
"Sale Type (Grouped)",
"Sales Channel",
"Sales Program",
"Siemens Probability"
FROM CRMCL_TOPICAREA.OSC."OLIs" 
WHERE "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
oli_frame = pd.read_sql(oli_sql,con=conn)

### Leads

In [38]:
leads_sql = '''
SELECT
"ACCOUNT_ID",
"ASSET_ID",
"Is Partner (Flag)",
"LEAD_ID",
"Last Automatic Assignment Date",
"Lead Age in Days",
"Lead Origin",
"Lead.Close Date",
"Lead.Contact Email",
"Lead.Contact First Name",
"Lead.Contact Job Role",
"Lead.Contact Job Title",
"Lead.Contact Last Name",
"Lead.Contact Postal Code",
"Lead.Country",
"Lead.Created Timestamp",
"Lead.FL Number",
"Lead.IB Vendor",
"Lead.Last Updated Timestamp",
"Lead.Opportunity Type",
"Lead.Sale Type",
"Lead.Sales Product Line",
"Lead.Status",
"PRODUCT_ID"
FROM CRMCL_TOPICAREA.OSC."Leads" 
WHERE "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
leads_frame = pd.read_sql(leads_sql,con=conn)

### Quotes In-Vitro

In [39]:
qvitro_sql = '''
SELECT
header."ACCOUNT_ID",
header."CPQ.Last Updated Timestamp",
header."CPR (Flag)",
header."DSO (Quote)",
header."DXCON.Approval Level",
header."DXCON.CPR Deal (Flag)",
header."DXCON.Configuration Approved (Flag)",
header."DXCON.Deal Billing Option",
header."DXCON.Deal Finance Option",
header."DXCON.Deal Finance Type",
header."DXCON.Deal Locked (Flag)",
header."DXCON.Last Saved Timestamp",
header."DXCON.Manual Discount (Flag)",
header."DXCON.Pricing Approved (Flag)",
header."DXCON.Sales Contract Duration (in months)",
header."Data Issue Comment",
header."Data Quality Issue (Flag)",
header."Has Automation (Flag)",
header."Has Instrument (Flag)",
header."Implausible Cost (Flag)",
header."Implausible Revenue (Flag)",
header."OPPORTUNITY_ID",
header."Opportunity Expired (Flag)",
header."Possible Test Quote (Flag)",
header."Quote Active (Flag)",
header."Quote Number and Revision",
header."Quote Status",
header."Quote Type",
header."Quote.Business Type",
header."Quote.Created Timestamp",
header."Quote.ISO",
header."Tender Type",
SUM(kpis."Contract Total Product Cost (GC)") AS "Total Cost (GC)",
SUM(kpis."Contract Total Revenue (GC)") AS "Total Revenue (GC)"
FROM CRMCL_TOPICAREA.CPQ.ACTIVE_QUOTES_HEADER header
LEFT JOIN CRMCL_TOPICAREA.CPQ."IN_VITRO_Contract_KPIs" kpis ON header."Quote Number and Revision"=kpis."Quote Number and Revision" 
WHERE header."Quote Type" = 'IN_VITRO'
AND header."ACCOUNT_ID" IN (
{0}
)
GROUP BY 
header."ACCOUNT_ID",
header."CPQ.Last Updated Timestamp",
header."CPR (Flag)",
header."DSO (Quote)",
header."DXCON.Approval Level",
header."DXCON.CPR Deal (Flag)",
header."DXCON.Configuration Approved (Flag)",
header."DXCON.Deal Billing Option",
header."DXCON.Deal Finance Option",
header."DXCON.Deal Finance Type",
header."DXCON.Deal Locked (Flag)",
header."DXCON.Last Saved Timestamp",
header."DXCON.Manual Discount (Flag)",
header."DXCON.Pricing Approved (Flag)",
header."DXCON.Sales Contract Duration (in months)",
header."Data Issue Comment",
header."Data Quality Issue (Flag)",
header."Has Automation (Flag)",
header."Has Instrument (Flag)",
header."Implausible Cost (Flag)",
header."Implausible Revenue (Flag)",
header."OPPORTUNITY_ID",
header."Opportunity Expired (Flag)",
header."Possible Test Quote (Flag)",
header."Quote Active (Flag)",
header."Quote Number and Revision",
header."Quote Status",
header."Quote Type",
header."Quote.Business Type",
header."Quote.Created Timestamp",
header."Quote.ISO",
header."Tender Type"
'''.format(acc_subset_)
qvitro_frame = pd.read_sql(qvitro_sql,con=conn)

### Quotes In-Vivo

In [40]:
qvivo_sql = '''
SELECT
header."ACCOUNT_ID",
header."CPQ.Last Updated Timestamp",
header."CPR (Flag)",
header."DSO (Quote)",
header."DXCON.Approval Level",
header."DXCON.CPR Deal (Flag)",
header."DXCON.Configuration Approved (Flag)",
header."DXCON.Deal Billing Option",
header."DXCON.Deal Finance Option",
header."DXCON.Deal Finance Type",
header."DXCON.Deal Locked (Flag)",
header."DXCON.Last Saved Timestamp",
header."DXCON.Manual Discount (Flag)",
header."DXCON.Pricing Approved (Flag)",
header."DXCON.Sales Contract Duration (in months)",
header."Data Issue Comment",
header."Data Quality Issue (Flag)",
header."Has Automation (Flag)",
header."Has Instrument (Flag)",
header."Implausible Cost (Flag)",
header."Implausible Revenue (Flag)",
header."OPPORTUNITY_ID",
header."Opportunity Expired (Flag)",
header."Possible Test Quote (Flag)",
header."Quote Active (Flag)",
header."Quote Number and Revision",
header."Quote Status",
header."Quote Type",
header."Quote.Business Type",
header."Quote.Created Timestamp",
header."Quote.ISO",
header."Tender Type",
SUM(costs."Amount (GC)"*costs."Quantity") AS "Total Cost (GC)",
SUM(rev."Quote Ext. Net Price (GC)"*rev."Quantity") AS "Total Revenue (GC)"
FROM CRMCL_TOPICAREA.CPQ.ACTIVE_QUOTES_HEADER header
LEFT JOIN CRMCL_TOPICAREA.CPQ."IN_VIVO_Quote_Details" rev ON header."Quote Number and Revision"=rev."Quote Number and Revision" 
LEFT JOIN CRMCL_TOPICAREA.CPQ."IN_VIVO_QuoteItem_Costs" costs ON header."Quote Number and Revision" = costs."Quote Number and Revision" 
WHERE header."Quote Type" = 'IN_VIVO'
AND header."ACCOUNT_ID" IN (
{0}
)
GROUP BY 
header."ACCOUNT_ID",
header."CPQ.Last Updated Timestamp",
header."CPR (Flag)",
header."DSO (Quote)",
header."DXCON.Approval Level",
header."DXCON.CPR Deal (Flag)",
header."DXCON.Configuration Approved (Flag)",
header."DXCON.Deal Billing Option",
header."DXCON.Deal Finance Option",
header."DXCON.Deal Finance Type",
header."DXCON.Deal Locked (Flag)",
header."DXCON.Last Saved Timestamp",
header."DXCON.Manual Discount (Flag)",
header."DXCON.Pricing Approved (Flag)",
header."DXCON.Sales Contract Duration (in months)",
header."Data Issue Comment",
header."Data Quality Issue (Flag)",
header."Has Automation (Flag)",
header."Has Instrument (Flag)",
header."Implausible Cost (Flag)",
header."Implausible Revenue (Flag)",
header."OPPORTUNITY_ID",
header."Opportunity Expired (Flag)",
header."Possible Test Quote (Flag)",
header."Quote Active (Flag)",
header."Quote Number and Revision",
header."Quote Status",
header."Quote Type",
header."Quote.Business Type",
header."Quote.Created Timestamp",
header."Quote.ISO",
header."Tender Type"
'''.format(acc_subset_)
qvivo_frame = pd.read_sql(qvivo_sql,con=conn)

### Quotes Services

In [41]:
qservice_sql = '''
SELECT
header."ACCOUNT_ID",
header."CPQ.Last Updated Timestamp",
header."CPR (Flag)",
header."DSO (Quote)",
header."DXCON.Approval Level",
header."DXCON.CPR Deal (Flag)",
header."DXCON.Configuration Approved (Flag)",
header."DXCON.Deal Billing Option",
header."DXCON.Deal Finance Option",
header."DXCON.Deal Finance Type",
header."DXCON.Deal Locked (Flag)",
header."DXCON.Last Saved Timestamp",
header."DXCON.Manual Discount (Flag)",
header."DXCON.Pricing Approved (Flag)",
header."DXCON.Sales Contract Duration (in months)",
header."Data Issue Comment",
header."Data Quality Issue (Flag)",
header."Has Automation (Flag)",
header."Has Instrument (Flag)",
header."Implausible Cost (Flag)",
header."Implausible Revenue (Flag)",
header."OPPORTUNITY_ID",
header."Opportunity Expired (Flag)",
header."Possible Test Quote (Flag)",
header."Quote Active (Flag)",
header."Quote Number and Revision",
header."Quote Status",
header."Quote Type",
header."Quote.Business Type",
header."Quote.Created Timestamp",
header."Quote.ISO",
header."Tender Type",
0 AS "Total Cost (GC)",
0 AS "Total Revenue (GC)"
FROM CRMCL_TOPICAREA.CPQ.ACTIVE_QUOTES_HEADER header
LEFT JOIN CRMCL_TOPICAREA.CPQ."IN_VIVO_Quote_Details" rev ON header."Quote Number and Revision"=rev."Quote Number and Revision" 
LEFT JOIN CRMCL_TOPICAREA.CPQ."IN_VIVO_QuoteItem_Costs" costs ON header."Quote Number and Revision" = costs."Quote Number and Revision" 
WHERE header."Quote Type" = 'SERVICES'
AND header."ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
qservice_frame = pd.read_sql(qservice_sql,con=conn)

### Installed Base from OSC.Assets

In [42]:
ib_asset_sql = '''
SELECT DISTINCT
"ACCOUNT_ID",
"ACCOUNT_ID (End User)",
"ASSET_ID",
"Asset Age (days)",
"Asset Age (months)",
"Asset Age (years)",
"Asset Equipment Eos Date",
"Asset FL Number",
"Asset Install Year",
"Asset Last Service Activity Date",
"Asset Material Number",
"Asset Service Contract End Date",
"Asset Service Contract Start Date",
"Asset Ship-to Account (SAP)",
"Asset Shipped Date",
"Asset System EoS Date",
"Asset Waranty End Date",
"Asset.Business Area",
"Asset.Business Line",
"Asset.Contract (Flag)",
"Asset.Created Timestamp",
"Asset.Last Updated Timestamp",
"Asset.Sales Product Line",
"Asset.Status",
"Competitor Asset (Flag)",
"EoS next 2 years & no Oppty (Flag)",
"EoS planned & no Oppty or no Strategy (Flag)",
"EoS reached & no Oppty (Flag)",
"Equipment Number",
"Evolve Capable (Flag)",
"IB Vendor Name",
"IBR - Expected Replacement Date",
"MTDA (Flag)",
"OLI_ID (Created by)",
"PRODUCT_ID"
FROM CRMCL_TOPICAREA.OSC."Assets" 
WHERE "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
ib_asset_frame = pd.read_sql(ib_asset_sql,con=conn)

### Installed Base from Smart Data Toolbox (SDTB, Services Data)

In [43]:
ib_sdtb_sql = '''
SELECT 
"ACCOUNT_ID",
"Asset FL Number",
YEAR("Asset Installation Date") AS "Asset Install Year",
"Asset Material Number",
"Asset Service Contract End Date",
"Asset Service Contract Start Date",
"Asset System EoS Date",
"Asset Waranty End Date",
"Last Updated Date" AS "Asset.Last Updated Timestamp",
"Equipment Status" AS "Asset.Status",
"Equipment Number",
"IB Product" AS "PRODUCT_ID"
FROM CRMCL_TOPICAREA.SDTB.SDTB_IBD 
WHERE "Deleted Asset (Flag)" <> 'Y' 
AND "Demo (Flag)" <> 'Y'
AND "ACCOUNT_ID" IN (
{0}
)
'''.format(acc_subset_)
ib_sdtb_frame = pd.read_sql(ib_sdtb_sql,con=conn)

### GCR-based Revenue and New Orders Report

In [44]:
gcr_sql = '''
SELECT 
"ACCOUNT_ID",
"ACCOUNT_ID (Payer)",
"ACCOUNT_ID (ShipTo)",
"ACCOUNT_ID (SoldTo)",
"Business Area",
"Business Line",
"Business Type",
"Cross Div. Business",
"Diagnostic Parameter",
"Disease State",
"FY",
"FY Period",
"GCR Reporting Date",
"ISO",
"Product Line",
"Product/Global Material Number",
"Sales Document Number HQ",
"Sales Document Number RC",
"Ship-to ID (SAP)",
"Sold-to ID (SAP)",
SUM("New Orders Outside Healthcare (GC PY Comparable)") AS "New Orders Outside Healthcare (GC PY Comparable)",
SUM("New Orders Outside Healthcare (GC)") AS "New Orders Outside Healthcare (GC)",
SUM("Revenue Outside Healthcare (GC PY Comparable)") AS "Revenue Outside Healthcare (GC PY Comparable)",
SUM("Revenue Outside Healthcare (GC)") AS "Revenue Outside Healthcare (GC)"
FROM CRMCL_TOPICAREA.GCR."Revenue_New_Orders" 
WHERE "ACCOUNT_ID" IN (
{0}
)
GROUP BY 
"ACCOUNT_ID",
"ACCOUNT_ID (Payer)",
"ACCOUNT_ID (ShipTo)",
"ACCOUNT_ID (SoldTo)",
"Business Area",
"Business Line",
"Business Type",
"Cross Div. Business",
"Diagnostic Parameter",
"Disease State",
"FY",
"FY Period",
"GCR Reporting Date",
"ISO",
"Product Line",
"Product/Global Material Number",
"Sales Document Number HQ",
"Sales Document Number RC",
"Ship-to ID (SAP)",
"Sold-to ID (SAP)"
'''.format(acc_subset_)
gcr_frame = pd.read_sql(gcr_sql,con=conn)

### Orders Data

In [45]:
orders_sql = '''
SELECT
qheader."ACCOUNT_ID",
header."Active (Flag)",
header."ISO",
header."Order Date",
header."Order Status",
header."Quote Number and Revision",
header."Sales Document Number RC",
header."Type",
CAST(det."ACCOUNT_ID (Bill-To)" AS CHAR(15)) AS "ACCOUNT_ID (Bill-To)",
CAST(det."ACCOUNT_ID (End-User)" AS CHAR(15)) AS "ACCOUNT_ID (End-User)",
CAST(det."ACCOUNT_ID (Pay-To)" AS CHAR(15)) AS "ACCOUNT_ID (Pay-To)",
CAST(det."ACCOUNT_ID (Ship-To)" AS CHAR(15)) AS "ACCOUNT_ID (Ship-To)",
CAST(det."ACCOUNT_ID (Sold-To)" AS CHAR(15)) AS "ACCOUNT_ID (Sold-To)",
det."ERP Order Number",
det."Order Number",
det."SAP Order Number",
det."Order Sale Type",
det."Quote Number and Revision",
SUM(det."Order Total Net Value") AS "Order Total Net Value",
SUM(det."Order Quantity") AS "Order Quantity"
FROM CRMCL_TOPICAREA.CPQ."Order_Details" det 
LEFT JOIN CRMCL_TOPICAREA.CPQ.ACTIVE_QUOTES_HEADER qheader ON det."Quote Number and Revision" = qheader."Quote Number and Revision" 
LEFT JOIN CRMCL_TOPICAREA.CPQ."Orders" header ON det."Quote Number and Revision" = header."Quote Number and Revision" 
WHERE qheader."ACCOUNT_ID" IN (
{0}
)
GROUP BY
qheader."ACCOUNT_ID",
header."Active (Flag)",
header."ISO",
header."Order Date",
header."Order Status",
header."Quote Number and Revision",
header."Sales Document Number RC",
header."Type",
"ACCOUNT_ID (Bill-To)",
"ACCOUNT_ID (End-User)",
"ACCOUNT_ID (Pay-To)",
"ACCOUNT_ID (Ship-To)",
"ACCOUNT_ID (Sold-To)",
det."ERP Order Number",
det."Order Number",
det."SAP Order Number",
det."Order Sale Type",
det."Quote Number and Revision"
'''.format(acc_subset_)
orders_frame = pd.read_sql(orders_sql,con=conn)

In [46]:
conn.close()

## Next Step: Preprocess and Build Model

Now that all entities are available, a definitive data key matrix has to be defined, and several entites have to be linked. 

Available Data Frames:
* acc_frame
* oppty_frame
* oli_frame
* leads_frame
* qvitro_frame
* qvivo_frame
* qservice_frame
* ib_asset_frame
* ib_sdtb_frame
* gcr_frame
* orders_frame

In [62]:
# Fix up quotes as one data frame
quotes_frame = pd.concat([qvitro_frame,qvivo_frame,qservice_frame],axis=0)

# Fix up IB from Assets and SDTB data as one data frame
ib_frame = pd.concat([ib_asset_frame,ib_sdtb_frame],axis=0)

Now available Data Frames, up for storage:
* acc_frame
* oppty_frame
* oli_frame
* leads_frame
* quotes_frame
* ib_frame
* gcr_frame
* orders_frame

In [66]:
frames = [
    acc_frame,
    oppty_frame,
    oli_frame,
    leads_frame,
    quotes_frame,
    ib_frame,
    gcr_frame,
    orders_frame
]

In [77]:
# Make a running index unique across all DB entities
c_ct = 0

#Unique DB index
for c_frame in frames:
    c_frame['DBID_ASSC'] = range(c_ct,len(c_frame)+c_ct)
    c_ct = c_frame['DBID_ASSC'].max()+1

## Final Step: Build a class

From this let's construct a class to make it available in other notebooks.

In [124]:
class CRMDB():
    
    def __init__(self,sql_path='./data/'):
        # Data init
        self.sql_path = sql_path
        self.frames = {}
        # Load frames
        self.load_dataframes()
        return
    
    def load_dataframes(self):
        # Make connection
        conn = snowflake.connector.connect(
            user='jan-lucas.deinhard@siemens-healthineers.com',
            account='shsitdl.west-europe.azure',
            authenticator='externalbrowser'
        )
        # Load accounts subset
        sqlQuery = self.load_sql('accID')
        self.acc_subset_ = '\''+'\',\''.join([str(k) for k in pd.read_sql(sqlQuery,con=conn)['ACCOUNT_ID'].tolist()])+'\''
        # List account files
        sql_files = [k for k in os.listdir(db.sql_path) if k.endswith('.sql')]
        sql_files.remove('accID.sql')
        
        # Load account details
        sqlQuery = self.load_sql('acc')
        self.frames['Accounts'] = pd.read_sql(sqlQuery.format(self.acc_subset_),con=conn)
        # Load Opptys
        sqlQuery = self.load_sql('oppty')
        self.frames['Opportunities'] = pd.read_sql(sqlQuery.format(self.acc_subset_),con=conn)
        # Load OLIs
        sqlQuery = self.load_sql('oli')
        self.frames['OLIs'] = pd.read_sql(sqlQuery.format(self.acc_subset_),con=conn)
        # Load Leads
        sqlQuery = self.load_sql('leads')
        self.frames['Leads'] = pd.read_sql(sqlQuery.format(self.acc_subset_),con=conn)
        # Load Quotes
        sqlQuery_vitro = self.load_sql('qvitro')
        sqlQuery_vivo = self.load_sql('qvivo')
        sqlQuery_service = self.load_sql('qservice')
        self.frames['Quotes'] = pd.concat(
            [
                pd.read_sql(sqlQuery_vitro.format(self.acc_subset_),con=conn),
                pd.read_sql(sqlQuery_vivo.format(self.acc_subset_),con=conn),
                pd.read_sql(sqlQuery_service.format(self.acc_subset_),con=conn)
            ],
        axis=0)
        # Close connection
        conn.close()
        return
    
    def load_sql(self,name):
        fd = open('{0}{1}.sql'.format(self.sql_path,name),'r')
        sqlQuery = fd.read()
        fd.close()
        return sqlQuery

In [130]:
sql_files = [k for k in os.listdir(db.sql_path) if k.endswith('.sql')]
sql_files.remove('accID.sql')

print(L)

['acc.sql', 'leads.sql', 'oli.sql', 'oppty.sql', 'qservice.sql', 'qvitro.sql', 'qvivo.sql']


In [125]:
db = CRMDB()

In [127]:
db.frames['Quotes']

,ACCOUNT_ID,CPQ.Last Updated Timestamp,CPR (Flag),DSO (Quote),DXCON.Approval Level,DXCON.CPR Deal (Flag),DXCON.Configuration Approved (Flag),DXCON.Deal Billing Option,DXCON.Deal Finance Option,DXCON.Deal Finance Type,...,Quote Active (Flag),Quote Number and Revision,Quote Status,Quote Type,Quote.Business Type,Quote.Created Timestamp,Quote.ISO,Tender Type,Total Cost (GC),Total Revenue (GC)
0,100000415399853,2017-06-16 00:00:00.000,N,N/A,N/A,N,N,N/A,N/A,N/A,...,Y,SU-1-KSK84X/0,N/A,IN_VITRO,N/A,2017-06-16 00:00:00.000,US,N/A,NaN,NaN
1,100000415399853,2019-06-06 00:00:00.000,N,N/A,N/A,N,N,N/A,Cash,Cash,...,Y,SU-1-OZX8JJ/6,N/A,IN_VITRO,N/A,2018-12-03 00:00:00.000,US,N/A,864942.015186,363483.215378
2,100001109135108,2017-03-03 00:00:00.000,N,N/A,N/A,N,N,N/A,Cash,Cash,...,Y,SE-1-N09WF9/0,N/A,IN_VITRO,N/A,2017-03-02 00:00:00.000,DE,N/A,1119.450000,1881.017870
3,100001109139774,2016-07-18 00:00:00.000,N,N/A,N/A,N,N,N/A,Lease,Direct Operating Lease,...,Y,SE-1-KK4IR7/2,N/A,IN_VITRO,N/A,2016-04-25 00:00:00.000,DE,N/A,118898.978571,393493.230000
4,100000040506349,2016-07-08 00:00:00.000,N,N/A,N/A,N,N,N/A,Cash,Cash,...,Y,SE-1-L5OPRL/0,N/A,IN_VITRO,N/A,2016-07-08 00:00:00.000,CH,N/A,649.875930,5648.740000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,100000415396119,1970-08-22 16:38:26.000,N,N/A,N/A,N,N,N/A,N/A,N/A,...,N,SU-1-QAZ5EX/0,SIGNED,IN_VIVO,N/A,NaT,US,N/A,NaN,0.000000
1210,100000415399381,1970-08-22 11:05:30.000,N,N/A,N/A,N,N,N/A,N/A,N/A,...,N,SU-1-JWLCD3/1,SIGNED,IN_VIVO,N/A,NaT,US,N/A,NaN,48190.470000
1211,100000417731583,1970-08-22 13:53:32.000,N,N/A,N/A,N,N,N/A,N/A,N/A,...,N,SU-1-OWYO1D/2,SIGNED,IN_VIVO,N/A,NaT,US,N/A,NaN,0.000000
1212,100000415397933,2020-03-16 18:25:10.670,N,N/A,N/A,N,N,N/A,N/A,N/A,...,Y,CPQ-141983/0,APPROVED,IN_VIVO,OWN_BUSINESS,2020-03-12 20:48:04.326,US,NON_TENDER,NaN,3993.345783
